<a href="https://colab.research.google.com/github/santiagonajera/Clasificacion_ABC_Python/blob/main/EAN_clase20Nov2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# URL del archivo Excel
url = 'https://github.com/santiagonajera/Clasificacion_ABC_Python/raw/main/ventas-2024-forecast.xlsx'

# Cargar los datos de las cuatro hojas del archivo Excel
lead_time = pd.read_excel(url, sheet_name='LeadTime-Dias')
historico = pd.read_excel(url, sheet_name='Historico')
forecast = pd.read_excel(url, sheet_name='Forecast')
precios_costos = pd.read_excel(url, sheet_name='Precios-Costos')

# Función personalizada para convertir columnas a tipo numérico
def convert_to_numeric(df, exclude_columns=['SKU']):
    for col in df.columns:
        if col not in exclude_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    return df

# Limpiar y preparar los datos
lead_time = convert_to_numeric(lead_time)
historico = convert_to_numeric(historico)
forecast = convert_to_numeric(forecast)
precios_costos = convert_to_numeric(precios_costos)

# Seleccionar solo las columnas numéricas para los cálculos posteriores
lead_time = lead_time.select_dtypes(include=[np.number])
historico = historico.select_dtypes(include=[np.number])
forecast = forecast.select_dtypes(include=[np.number])
precios_costos = precios_costos.select_dtypes(include=[np.number])

# Clasificación ABC
# Calcular el valor total multiplicando la suma de las ventas pronosticadas por el precio
forecast['Total_Value'] = forecast.sum(axis=1) * precios_costos['Precio']

# Calcular el porcentaje acumulado
forecast['Cumulative_Percentage'] = forecast['Total_Value'].cumsum() / forecast['Total_Value'].sum() * 100

# Asignar categorías A (0-60%), B (60-80%), C (80-100%)
def assign_abc_category(percentage):
    if percentage <= 60:
        return 'A'
    elif percentage <= 80:
        return 'B'
    else:
        return 'C'

forecast['ABC_Category'] = forecast['Cumulative_Percentage'].apply(assign_abc_category)

# Mostrar el resultado final
print(forecast[['SKU', 'Total_Value', 'Cumulative_Percentage', 'ABC_Category']])


KeyError: "['SKU'] not in index"

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Función para convertir columnas a tipo numérico y llenar NaN con 0
def convert_to_numeric_and_fillna(column):
    return pd.to_numeric(column, errors='coerce').fillna(0)

# Cargar los datos de las cuatro hojas del archivo Excel
url = "https://github.com/santiagonajera/Clasificacion_ABC_Python/raw/main/ventas-2024-forecast.xlsx"

# Cargar cada hoja en un DataFrame
df_leadtime = pd.read_excel(url, sheet_name='LeadTime-Dias')
df_historico = pd.read_excel(url, sheet_name='Historico')
df_forecast = pd.read_excel(url, sheet_name='Forecast')
df_precios = pd.read_excel(url, sheet_name='Precios-Costos')

# Limpiar y preparar los datos
# Convertir columnas a tipo numérico y llenar NaN con 0
for df in [df_leadtime, df_historico, df_forecast, df_precios]:
    for col in df.columns:
        if col != 'SKU':
            df[col] = convert_to_numeric_and_fillna(df[col])

# Seleccionar solo las columnas numéricas para los cálculos posteriores
df_leadtime = df_leadtime.select_dtypes(include=[np.number])
df_historico = df_historico.select_dtypes(include=[np.number])
df_forecast = df_forecast.select_dtypes(include=[np.number])
df_precios = df_precios.select_dtypes(include=[np.number])

# Clasificación ABC
# Calcular el valor total multiplicando la suma de las ventas pronosticadas por el precio
df_forecast['Total_Value'] = df_forecast.sum(axis=1) * df_precios.iloc[:, 0]

# Calcular el porcentaje acumulado
df_forecast['Cumulative_Percentage'] = df_forecast['Total_Value'].cumsum() / df_forecast['Total_Value'].sum() * 100

# Asignar categorías ABC
df_forecast['ABC_Category'] = pd.cut(df_forecast['Cumulative_Percentage'], bins=[0, 60, 80, 100], labels=['A', 'B', 'C'])

# Clasificación XYZ
# Calcular la desviación estándar de las ventas históricas
df_historico['Std_Dev'] = df_historico.std(axis=1)

# Calcular el coeficiente de variación
df_historico['CV'] = df_historico['Std_Dev'] / df_historico.mean(axis=1)

# Asignar categorías XYZ
df_historico['XYZ_Category'] = pd.cut(df_historico['CV'], bins=[0, 0.1, 0.25, np.inf], labels=['X', 'Y', 'Z'])

# Combinar las clasificaciones ABC y XYZ
df_combined = df_forecast[['SKU', 'ABC_Category']].merge(df_historico[['SKU', 'XYZ_Category']], on='SKU')

# Optimización de inventarios (ejemplo básico)
# Calcular el stock de seguridad basado en la clasificación ABC-XYZ
df_combined['Stock_Safety'] = df_combined.apply(lambda row:
    df_leadtime.loc[df_leadtime['SKU'] == row['SKU'], 'LeadTime'].values[0] * df_historico.loc[df_historico['SKU'] == row['SKU'], 'Std_Dev'].values[0] *
    (1 + (row['ABC_Category'] == 'A') * 0.5 + (row['ABC_Category'] == 'B') * 0.3 + (row['ABC_Category'] == 'C') * 0.1), axis=1)

# Guardar los resultados en un archivo Excel
with pd.ExcelWriter('resultados_clasificacion_optimizacion.xlsx') as writer:
    df_combined.to_excel(writer, sheet_name='Clasificacion_ABC_XYZ', index=False)
    df_combined[['SKU', 'Stock_Safety']].to_excel(writer, sheet_name='Optimizacion_Inventario', index=False)

print("Proceso completado. Los resultados se han guardado en 'resultados_clasificacion_optimizacion.xlsx'.")

KeyError: "['SKU'] not in index"

In [4]:
import pandas as pd
import numpy as np
from scipy import stats

def convert_to_numeric(df, exclude_cols=['SKU']):
    """
    Convierte todas las columnas a numéricas, excepto las especificadas en exclude_cols
    """
    for col in df.columns:
        if col not in exclude_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    return df

def load_and_clean_data(url):
    """
    Carga los datos de todas las hojas y las limpia
    """
    # Cargar todas las hojas
    lead_time = pd.read_excel(url, sheet_name='LeadTime-Dias')
    historico = pd.read_excel(url, sheet_name='Historico')
    forecast = pd.read_excel(url, sheet_name='Forecast')
    precios = pd.read_excel(url, sheet_name='Precios-Costos')

    # Limpiar cada DataFrame
    lead_time = convert_to_numeric(lead_time)
    historico = convert_to_numeric(historico)
    forecast = convert_to_numeric(forecast)
    precios = convert_to_numeric(precios)

    return lead_time, historico, forecast, precios

def clasificacion_abc(forecast_df, precios_df):
    """
    Realiza la clasificación ABC basada en el valor total de ventas
    """
    # Calcular ventas totales pronosticadas por SKU
    ventas_forecast = forecast_df.set_index('SKU').sum(axis=1)

    # Obtener precios por SKU
    precios = precios_df.set_index('SKU')['Precio']

    # Calcular valor total
    valor_total = ventas_forecast * precios

    # Ordenar de mayor a menor
    valor_total_sorted = valor_total.sort_values(ascending=False)

    # Calcular porcentaje acumulado
    porcentaje_acumulado = (valor_total_sorted.cumsum() / valor_total_sorted.sum()) * 100

    # Asignar categorías ABC
    def get_categoria_abc(porcentaje):
        if porcentaje <= 60:
            return 'A'
        elif porcentaje <= 80:
            return 'B'
        else:
            return 'C'

    categorias_abc = porcentaje_acumulado.apply(get_categoria_abc)

    # Crear DataFrame con resultados
    resultados_abc = pd.DataFrame({
        'SKU': categorias_abc.index,
        'Valor_Total': valor_total_sorted,
        'Porcentaje_Acumulado': porcentaje_acumulado,
        'Categoria_ABC': categorias_abc
    })

    return resultados_abc

def main():
    # URL del archivo
    url = "https://github.com/santiagonajera/Clasificacion_ABC_Python/raw/main/ventas-2024-forecast.xlsx"

    # Cargar y limpiar datos
    lead_time, historico, forecast, precios = load_and_clean_data(url)

    # Realizar clasificación ABC
    resultados_abc = clasificacion_abc(forecast, precios)

    return resultados_abc

if __name__ == "__main__":
    resultados = main()

In [5]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# URL del archivo Excel
url = 'https://github.com/santiagonajera/Clasificacion_ABC_Python/raw/main/ventas-2024-forecast.xlsx'

# Cargar los datos de las cuatro hojas del archivo Excel
lead_time = pd.read_excel(url, sheet_name='LeadTime-Dias')
historico = pd.read_excel(url, sheet_name='Historico')
forecast = pd.read_excel(url, sheet_name='Forecast')
precios_costos = pd.read_excel(url, sheet_name='Precios-Costos')

# Función personalizada para convertir columnas a tipo numérico
def convert_to_numeric(df, exclude_columns=['SKU']):
    for col in df.columns:
        if col not in exclude_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    return df

# Limpiar y preparar los datos
lead_time = convert_to_numeric(lead_time)
historico = convert_to_numeric(historico)
forecast = convert_to_numeric(forecast)
precios_costos = convert_to_numeric(precios_costos)

# Seleccionar solo las columnas numéricas para los cálculos posteriores, manteniendo 'SKU'
lead_time_numeric = lead_time.select_dtypes(include=[np.number])
historico_numeric = historico.select_dtypes(include=[np.number])
forecast_numeric = forecast.select_dtypes(include=[np.number])
precios_costos_numeric = precios_costos.select_dtypes(include=[np.number])

# Clasificación ABC
# Calcular el valor total multiplicando la suma de las ventas pronosticadas por el precio
forecast['Total_Value'] = forecast_numeric.sum(axis=1) * precios_costos['Precio']

# Calcular el porcentaje acumulado
forecast['Cumulative_Percentage'] = forecast['Total_Value'].cumsum() / forecast['Total_Value'].sum() * 100

# Asignar categorías A (0-60%), B (60-80%), C (80-100%)
def assign_abc_category(percentage):
    if percentage <= 60:
        return 'A'
    elif percentage <= 80:
        return 'B'
    else:
        return 'C'

forecast['ABC_Category'] = forecast['Cumulative_Percentage'].apply(assign_abc_category)

# Mostrar el resultado final
print(forecast[['SKU', 'Total_Value', 'Cumulative_Percentage', 'ABC_Category']])


       SKU  Total_Value  Cumulative_Percentage ABC_Category
0    SKU 1   2056600.61               3.830092            A
1    SKU 2   3725423.60              10.768101            A
2    SKU 3   3047630.92              16.443829            A
3    SKU 4   2973709.29              21.981889            A
4    SKU 5   1620510.82              24.999833            A
5    SKU 6   1768272.66              28.292960            A
6    SKU 7   1957194.51              31.937923            A
7    SKU 8    933311.16              33.676067            A
8    SKU 9   1691464.32              36.826150            A
9   SKU 10   1928633.76              40.417924            A
10  SKU 11   2645470.81              45.344693            A
11  SKU 12   2666759.61              50.311108            A
12  SKU 13   2103481.71              54.228508            A
13  SKU 14   2489953.92              58.865652            A
14  SKU 15   1191377.52              61.084403            B
15  SKU 16   1431411.03              63.

In [6]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

# Función para convertir columnas a tipo numérico y llenar NaN con 0
def convert_to_numeric_and_fillna(column):
    return pd.to_numeric(column, errors='coerce').fillna(0)

# Cargar los datos de las cuatro hojas del archivo Excel
url = "https://github.com/santiagonajera/Clasificacion_ABC_Python/raw/main/ventas-2024-forecast.xlsx"

# Cargar cada hoja en un DataFrame
df_leadtime = pd.read_excel(url, sheet_name='LeadTime-Dias')
df_historico = pd.read_excel(url, sheet_name='Historico')
df_forecast = pd.read_excel(url, sheet_name='Forecast')
df_precios = pd.read_excel(url, sheet_name='Precios-Costos')

# Limpiar y preparar los datos
# Convertir columnas a tipo numérico y llenar NaN con 0
for df in [df_leadtime, df_historico, df_forecast, df_precios]:
    for col in df.columns:
        if col != 'SKU':
            df[col] = convert_to_numeric_and_fillna(df[col])

# Mantener la columna SKU en los DataFrames
df_leadtime_sku = df_leadtime[['SKU']]
df_historico_sku = df_historico[['SKU']]
df_forecast_sku = df_forecast[['SKU']]
df_precios_sku = df_precios[['SKU']]

# Seleccionar solo las columnas numéricas para los cálculos posteriores
df_leadtime = df_leadtime.select_dtypes(include=[np.number])
df_historico = df_historico.select_dtypes(include=[np.number])
df_forecast = df_forecast.select_dtypes(include=[np.number])
df_precios = df_precios.select_dtypes(include=[np.number])

# Agregar la columna SKU nuevamente a los DataFrames
df_leadtime = pd.concat([df_leadtime_sku, df_leadtime], axis=1)
df_historico = pd.concat([df_historico_sku, df_historico], axis=1)
df_forecast = pd.concat([df_forecast_sku, df_forecast], axis=1)
df_precios = pd.concat([df_precios_sku, df_precios], axis=1)

# Clasificación ABC
# Calcular el valor total multiplicando la suma de las ventas pronosticadas por el precio
df_forecast['Total_Value'] = df_forecast.iloc[:, 1:].sum(axis=1) * df_precios.iloc[:, 1]

# Calcular el porcentaje acumulado
df_forecast['Cumulative_Percentage'] = df_forecast['Total_Value'].cumsum() / df_forecast['Total_Value'].sum() * 100

# Asignar categorías ABC
df_forecast['ABC_Category'] = pd.cut(df_forecast['Cumulative_Percentage'], bins=[0, 60, 80, 100], labels=['A', 'B', 'C'])

# Clasificación XYZ
# Calcular la desviación estándar de las ventas históricas
df_historico['Std_Dev'] = df_historico.iloc[:, 1:].std(axis=1)

# Calcular el coeficiente de variación
df_historico['CV'] = df_historico['Std_Dev'] / df_historico.iloc[:, 1:].mean(axis=1)

# Asignar categorías XYZ
df_historico['XYZ_Category'] = pd.cut(df_historico['CV'], bins=[0, 0.1, 0.25, np.inf], labels=['X', 'Y', 'Z'])

# Combinar las clasificaciones ABC y XYZ
df_combined = df_forecast[['SKU', 'ABC_Category']].merge(df_historico[['SKU', 'XYZ_Category']], on='SKU')

# Optimización de inventarios (ejemplo básico)
# Calcular el stock de seguridad basado en la clasificación ABC-XYZ
df_combined['Stock_Safety'] = df_combined.apply(lambda row:
    df_leadtime.loc[df_leadtime['SKU'] == row['SKU'], 'LeadTime'].values[0] * df_historico.loc[df_historico['SKU'] == row['SKU'], 'Std_Dev'].values[0] *
    (1 + (row['ABC_Category'] == 'A') * 0.5 + (row['ABC_Category'] == 'B') * 0.3 + (row['ABC_Category'] == 'C') * 0.1), axis=1)

# Guardar los resultados en un archivo Excel
with pd.ExcelWriter('resultados_clasificacion_optimizacion.xlsx') as writer:
    df_combined.to_excel(writer, sheet_name='Clasificacion_ABC_XYZ', index=False)
    df_combined[['SKU', 'Stock_Safety']].to_excel(writer, sheet_name='Optimizacion_Inventario', index=False)

print("Proceso completado. Los resultados se han guardado en 'resultados_clasificacion_optimizacion.xlsx'.")

KeyError: 'LeadTime'

In [7]:
import pandas as pd
import numpy as np
from scipy import stats

def convert_to_numeric(df, exclude_cols=['SKU']):
    """
    Convierte todas las columnas a numéricas, excepto las especificadas en exclude_cols
    """
    for col in df.columns:
        if col not in exclude_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    return df

def load_and_clean_data(url):
    """
    Carga los datos de todas las hojas y las limpia
    """
    # Cargar todas las hojas
    lead_time = pd.read_excel(url, sheet_name='LeadTime-Dias')
    historico = pd.read_excel(url, sheet_name='Historico')
    forecast = pd.read_excel(url, sheet_name='Forecast')
    precios = pd.read_excel(url, sheet_name='Precios-Costos')

    # Limpiar cada DataFrame
    lead_time = convert_to_numeric(lead_time)
    historico = convert_to_numeric(historico)
    forecast = convert_to_numeric(forecast)
    precios = convert_to_numeric(precios)

    return lead_time, historico, forecast, precios

def clasificacion_abc(forecast_df, precios_df):
    """
    Realiza la clasificación ABC basada en el valor total de ventas
    """
    # Calcular ventas totales pronosticadas por SKU
    ventas_forecast = forecast_df.set_index('SKU').sum(axis=1)

    # Obtener precios por SKU
    precios = precios_df.set_index('SKU')['Precio']

    # Calcular valor total
    valor_total = ventas_forecast * precios

    # Ordenar de mayor a menor
    valor_total_sorted = valor_total.sort_values(ascending=False)

    # Calcular porcentaje acumulado
    porcentaje_acumulado = (valor_total_sorted.cumsum() / valor_total_sorted.sum()) * 100

    # Asignar categorías ABC
    def get_categoria_abc(porcentaje):
        if porcentaje <= 60:
            return 'A'
        elif porcentaje <= 80:
            return 'B'
        else:
            return 'C'

    categorias_abc = porcentaje_acumulado.apply(get_categoria_abc)

    # Crear DataFrame con resultados
    resultados_abc = pd.DataFrame({
        'SKU': categorias_abc.index,
        'Valor_Total': valor_total_sorted,
        'Porcentaje_Acumulado': porcentaje_acumulado,
        'Categoria_ABC': categorias_abc
    })

    return resultados_abc

def mostrar_resultados(resultados_abc):
    """
    Muestra los resultados de la clasificación ABC
    """
    # Formatear Valor_Total y Porcentaje_Acumulado
    resultados_abc['Valor_Total'] = resultados_abc['Valor_Total'].round(2)
    resultados_abc['Porcentaje_Acumulado'] = resultados_abc['Porcentaje_Acumulado'].round(2)

    # Mostrar los primeros registros
    print("\nPrimeros 10 registros de la clasificación ABC:")
    print(resultados_abc.head(10).to_string())

    # Mostrar resumen por categoría
    print("\nResumen por categoría ABC:")
    resumen = resultados_abc.groupby('Categoria_ABC').agg({
        'SKU': 'count',
        'Valor_Total': 'sum'
    }).round(2)

    resumen['Porcentaje_Valor'] = (resumen['Valor_Total'] / resumen['Valor_Total'].sum() * 100).round(2)
    resumen['Porcentaje_Items'] = (resumen['SKU'] / resumen['SKU'].sum() * 100).round(2)

    print(resumen)

def main():
    # URL del archivo
    url = "https://github.com/santiagonajera/Clasificacion_ABC_Python/raw/main/ventas-2024-forecast.xlsx"

    # Cargar y limpiar datos
    lead_time, historico, forecast, precios = load_and_clean_data(url)

    # Realizar clasificación ABC
    resultados_abc = clasificacion_abc(forecast, precios)

    # Mostrar resultados
    mostrar_resultados(resultados_abc)

    return resultados_abc

if __name__ == "__main__":
    resultados = main()


Primeros 10 registros de la clasificación ABC:
           SKU  Valor_Total  Porcentaje_Acumulado Categoria_ABC
SKU                                                            
SKU 2    SKU 2   3725423.60                  6.94             A
SKU 3    SKU 3   3047630.92                 12.61             A
SKU 4    SKU 4   2973709.29                 18.15             A
SKU 12  SKU 12   2666759.61                 23.12             A
SKU 11  SKU 11   2645470.81                 28.04             A
SKU 14  SKU 14   2489953.92                 32.68             A
SKU 18  SKU 18   2483160.96                 37.31             A
SKU 17  SKU 17   2292565.00                 41.58             A
SKU 13  SKU 13   2103481.71                 45.49             A
SKU 1    SKU 1   2056600.61                 49.32             A

Resumen por categoría ABC:
               SKU  Valor_Total  Porcentaje_Valor  Porcentaje_Items
Categoria_ABC                                                      
A               13  

In [8]:
import pandas as pd
import numpy as np
from scipy import stats

def convert_to_numeric(df, exclude_cols=['SKU']):
    """
    Convierte todas las columnas a numéricas, excepto las especificadas en exclude_cols
    """
    for col in df.columns:
        if col not in exclude_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    return df

def load_and_clean_data(url):
    """
    Carga los datos de todas las hojas y las limpia
    """
    # Cargar todas las hojas
    lead_time = pd.read_excel(url, sheet_name='LeadTime-Dias')
    historico = pd.read_excel(url, sheet_name='Historico')
    forecast = pd.read_excel(url, sheet_name='Forecast')
    precios = pd.read_excel(url, sheet_name='Precios-Costos')

    # Limpiar cada DataFrame
    lead_time = convert_to_numeric(lead_time)
    historico = convert_to_numeric(historico)
    forecast = convert_to_numeric(forecast)
    precios = convert_to_numeric(precios)

    return lead_time, historico, forecast, precios

def clasificacion_abc(forecast_df, precios_df):
    """
    Realiza la clasificación ABC basada en el valor total de ventas
    """
    # Calcular ventas totales pronosticadas por SKU
    ventas_forecast = forecast_df.set_index('SKU').sum(axis=1)

    # Obtener precios por SKU
    precios = precios_df.set_index('SKU')['Precio']

    # Calcular valor total
    valor_total = ventas_forecast * precios

    # Ordenar de mayor a menor
    valor_total_sorted = valor_total.sort_values(ascending=False)

    # Calcular porcentaje acumulado
    porcentaje_acumulado = (valor_total_sorted.cumsum() / valor_total_sorted.sum()) * 100

    # Asignar categorías ABC
    def get_categoria_abc(porcentaje):
        if porcentaje <= 60:
            return 'A'
        elif porcentaje <= 80:
            return 'B'
        else:
            return 'C'

    categorias_abc = porcentaje_acumulado.apply(get_categoria_abc)

    # Crear DataFrame con resultados
    resultados_abc = pd.DataFrame({
        'SKU': categorias_abc.index,
        'Valor_Total': valor_total_sorted,
        'Porcentaje_Acumulado': porcentaje_acumulado,
        'Categoria_ABC': categorias_abc
    })

    return resultados_abc

def clasificacion_xyz(historico_df):
    """
    Realiza la clasificación XYZ basada en el coeficiente de variación
    """
    # Seleccionar solo las columnas numéricas y los últimos 18 meses
    cols_numericas = historico_df.select_dtypes(include=[np.number]).columns[-18:]
    datos_recientes = historico_df[['SKU'] + list(cols_numericas)]

    # Calcular coeficiente de variación
    cv = datos_recientes.set_index('SKU')[cols_numericas].apply(lambda x: x.std() / x.mean() * 100, axis=1)

    # Asignar categorías XYZ usando qcut
    categorias_xyz = pd.qcut(cv, q=3, labels=['X', 'Y', 'Z'])

    # Crear DataFrame con resultados
    resultados_xyz = pd.DataFrame({
        'SKU': cv.index,
        'Coef_Variacion': cv,
        'Categoria_XYZ': categorias_xyz
    })

    return resultados_xyz

def combinar_clasificaciones(abc_df, xyz_df):
    """
    Combina las clasificaciones ABC y XYZ y asigna niveles de servicio
    """
    # Combinar resultados
    resultados_combinados = abc_df.merge(xyz_df, on='SKU', how='inner')

    # Convertir categorías a strings y combinarlas
    resultados_combinados['Categoria_ABC_XYZ'] = (
        resultados_combinados['Categoria_ABC'].astype(str) +
        resultados_combinados['Categoria_XYZ'].astype(str)
    )

    # Definir niveles de servicio
    niveles_servicio = {
        'AX': 0.95, 'AY': 0.90, 'AZ': 0.85,
        'BX': 0.80, 'BY': 0.75, 'BZ': 0.70,
        'CX': 0.65, 'CY': 0.60, 'CZ': 0.55
    }

    # Asignar niveles de servicio
    resultados_combinados['Nivel_Servicio'] = (
        resultados_combinados['Categoria_ABC_XYZ'].map(niveles_servicio)
    )

    return resultados_combinados

def mostrar_resultados(resultados_combinados):
    """
    Muestra los resultados de la clasificación ABC-XYZ
    """
    # Formatear columnas numéricas
    resultados_combinados['Valor_Total'] = resultados_combinados['Valor_Total'].round(2)
    resultados_combinados['Porcentaje_Acumulado'] = resultados_combinados['Porcentaje_Acumulado'].round(2)
    resultados_combinados['Coef_Variacion'] = resultados_combinados['Coef_Variacion'].round(2)

    # Mostrar los primeros registros
    print("\nPrimeros 10 registros de la clasificación ABC-XYZ:")
    print(resultados_combinados.head(10).to_string())

    # Mostrar resumen por categoría combinada
    print("\nResumen por categoría ABC-XYZ:")
    resumen = resultados_combinados.groupby('Categoria_ABC_XYZ').agg({
        'SKU': 'count',
        'Valor_Total': 'sum',
        'Nivel_Servicio': 'first'
    }).round(2)

    resumen['Porcentaje_Items'] = (resumen['SKU'] / resumen['SKU'].sum() * 100).round(2)
    resumen['Porcentaje_Valor'] = (resumen['Valor_Total'] / resumen['Valor_Total'].sum() * 100).round(2)

    print(resumen)

def main():
    # URL del archivo
    url = "https://github.com/santiagonajera/Clasificacion_ABC_Python/raw/main/ventas-2024-forecast.xlsx"

    # Cargar y limpiar datos
    lead_time, historico, forecast, precios = load_and_clean_data(url)

    # Realizar clasificaciones
    resultados_abc = clasificacion_abc(forecast, precios)
    resultados_xyz = clasificacion_xyz(historico)

    # Combinar clasificaciones
    resultados_finales = combinar_clasificaciones(resultados_abc, resultados_xyz)

    # Mostrar resultados
    mostrar_resultados(resultados_finales)

    return resultados_finales

if __name__ == "__main__":
    resultados = main()

ValueError: 'SKU' is both an index level and a column label, which is ambiguous.

In [9]:
import pandas as pd
import numpy as np
from scipy import stats

def convert_to_numeric(df, exclude_cols=['SKU']):
    """
    Convierte todas las columnas a numéricas, excepto las especificadas en exclude_cols
    """
    for col in df.columns:
        if col not in exclude_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    return df

def load_and_clean_data(url):
    """
    Carga los datos de todas las hojas y las limpia
    """
    # Cargar todas las hojas
    lead_time = pd.read_excel(url, sheet_name='LeadTime-Dias')
    historico = pd.read_excel(url, sheet_name='Historico')
    forecast = pd.read_excel(url, sheet_name='Forecast')
    precios = pd.read_excel(url, sheet_name='Precios-Costos')

    # Limpiar cada DataFrame
    lead_time = convert_to_numeric(lead_time)
    historico = convert_to_numeric(historico)
    forecast = convert_to_numeric(forecast)
    precios = convert_to_numeric(precios)

    return lead_time, historico, forecast, precios

def clasificacion_abc(forecast_df, precios_df):
    """
    Realiza la clasificación ABC basada en el valor total de ventas
    """
    # Crear copia de los DataFrames para evitar modificar los originales
    forecast = forecast_df.copy()
    precios = precios_df.copy()

    # Calcular ventas totales pronosticadas por SKU
    forecast.set_index('SKU', inplace=True)
    ventas_forecast = forecast.sum(axis=1)

    # Obtener precios por SKU
    precios.set_index('SKU', inplace=True)
    precios_serie = precios['Precio']

    # Calcular valor total
    valor_total = ventas_forecast * precios_serie

    # Ordenar de mayor a menor
    valor_total_sorted = valor_total.sort_values(ascending=False)

    # Calcular porcentaje acumulado
    porcentaje_acumulado = (valor_total_sorted.cumsum() / valor_total_sorted.sum() * 100)

    # Asignar categorías ABC
    def get_categoria_abc(porcentaje):
        if porcentaje <= 60:
            return 'A'
        elif porcentaje <= 80:
            return 'B'
        else:
            return 'C'

    categorias_abc = porcentaje_acumulado.apply(get_categoria_abc)

    # Crear DataFrame con resultados
    resultados_abc = pd.DataFrame({
        'Valor_Total': valor_total_sorted,
        'Porcentaje_Acumulado': porcentaje_acumulado,
        'Categoria_ABC': categorias_abc
    })

    # Agregar SKU como columna
    resultados_abc['SKU'] = resultados_abc.index
    resultados_abc.reset_index(drop=True, inplace=True)

    return resultados_abc

def clasificacion_xyz(historico_df):
    """
    Realiza la clasificación XYZ basada en el coeficiente de variación
    """
    # Crear copia del DataFrame
    historico = historico_df.copy()

    # Seleccionar solo las columnas numéricas y los últimos 18 meses
    cols_numericas = historico.select_dtypes(include=[np.number]).columns[-18:]
    historico.set_index('SKU', inplace=True)

    # Calcular coeficiente de variación
    cv = historico[cols_numericas].apply(lambda x: x.std() / x.mean() * 100 if x.mean() != 0 else float('inf'), axis=1)

    # Asignar categorías XYZ usando qcut
    categorias_xyz = pd.qcut(cv, q=3, labels=['X', 'Y', 'Z'])

    # Crear DataFrame con resultados
    resultados_xyz = pd.DataFrame({
        'Coef_Variacion': cv,
        'Categoria_XYZ': categorias_xyz
    })

    # Agregar SKU como columna
    resultados_xyz['SKU'] = resultados_xyz.index
    resultados_xyz.reset_index(drop=True, inplace=True)

    return resultados_xyz

def combinar_clasificaciones(abc_df, xyz_df):
    """
    Combina las clasificaciones ABC y XYZ y asigna niveles de servicio
    """
    # Combinar resultados
    resultados_combinados = pd.merge(abc_df, xyz_df, on='SKU', how='inner')

    # Convertir categorías a strings y combinarlas
    resultados_combinados['Categoria_ABC_XYZ'] = (
        resultados_combinados['Categoria_ABC'].astype(str) +
        resultados_combinados['Categoria_XYZ'].astype(str)
    )

    # Definir niveles de servicio
    niveles_servicio = {
        'AX': 0.95, 'AY': 0.90, 'AZ': 0.85,
        'BX': 0.80, 'BY': 0.75, 'BZ': 0.70,
        'CX': 0.65, 'CY': 0.60, 'CZ': 0.55
    }

    # Asignar niveles de servicio
    resultados_combinados['Nivel_Servicio'] = (
        resultados_combinados['Categoria_ABC_XYZ'].map(niveles_servicio)
    )

    return resultados_combinados

def mostrar_resultados(resultados_combinados):
    """
    Muestra los resultados de la clasificación ABC-XYZ
    """
    # Formatear columnas numéricas
    resultados_combinados['Valor_Total'] = resultados_combinados['Valor_Total'].round(2)
    resultados_combinados['Porcentaje_Acumulado'] = resultados_combinados['Porcentaje_Acumulado'].round(2)
    resultados_combinados['Coef_Variacion'] = resultados_combinados['Coef_Variacion'].round(2)

    # Mostrar los primeros registros
    print("\nPrimeros 10 registros de la clasificación ABC-XYZ:")
    print(resultados_combinados.head(10).to_string())

    # Mostrar resumen por categoría combinada
    print("\nResumen por categoría ABC-XYZ:")
    resumen = resultados_combinados.groupby('Categoria_ABC_XYZ').agg({
        'SKU': 'count',
        'Valor_Total': 'sum',
        'Nivel_Servicio': 'first'
    }).round(2)

    resumen['Porcentaje_Items'] = (resumen['SKU'] / resumen['SKU'].sum() * 100).round(2)
    resumen['Porcentaje_Valor'] = (resumen['Valor_Total'] / resumen['Valor_Total'].sum() * 100).round(2)

    print(resumen)

def main():
    # URL del archivo
    url = "https://github.com/santiagonajera/Clasificacion_ABC_Python/raw/main/ventas-2024-forecast.xlsx"

    # Cargar y limpiar datos
    lead_time, historico, forecast, precios = load_and_clean_data(url)

    # Realizar clasificaciones
    resultados_abc = clasificacion_abc(forecast, precios)
    resultados_xyz = clasificacion_xyz(historico)

    # Combinar clasificaciones
    resultados_finales = combinar_clasificaciones(resultados_abc, resultados_xyz)

    # Mostrar resultados
    mostrar_resultados(resultados_finales)

    return resultados_finales

if __name__ == "__main__":
    resultados = main()


Primeros 10 registros de la clasificación ABC-XYZ:
   Valor_Total  Porcentaje_Acumulado Categoria_ABC     SKU  Coef_Variacion Categoria_XYZ Categoria_ABC_XYZ  Nivel_Servicio
0   3725423.60                  6.94             A   SKU 2           16.25             X                AX            0.95
1   3047630.92                 12.61             A   SKU 3           17.16             X                AX            0.95
2   2973709.29                 18.15             A   SKU 4           16.61             X                AX            0.95
3   2666759.61                 23.12             A  SKU 12           18.85             Y                AY            0.90
4   2645470.81                 28.04             A  SKU 11           16.81             X                AX            0.95
5   2489953.92                 32.68             A  SKU 14           18.62             Y                AY            0.90
6   2483160.96                 37.31             A  SKU 18           27.37             

In [10]:
import pandas as pd
import numpy as np
from scipy import stats

# [Mantener todas las funciones anteriores hasta combinar_clasificaciones() sin cambios]

def calcular_estadisticas_demanda(historico_df):
    """
    Calcula estadísticas de demanda usando los últimos 18 meses
    """
    # Seleccionar solo columnas numéricas de los últimos 18 meses
    columnas_numericas = historico_df.select_dtypes(include=[np.number]).columns[-18:]

    # Calcular estadísticas
    demanda_promedio = historico_df[columnas_numericas].mean(axis=1)
    demanda_std = historico_df[columnas_numericas].std(axis=1)

    return pd.DataFrame({
        'SKU': historico_df['SKU'],
        'Demanda_Promedio': demanda_promedio,
        'Demanda_Std': demanda_std
    })

def calcular_estadisticas_leadtime(leadtime_df):
    """
    Calcula estadísticas de lead time y convierte a meses
    """
    # Seleccionar solo columnas numéricas
    columnas_numericas = leadtime_df.select_dtypes(include=[np.number]).columns

    # Calcular estadísticas
    lt_promedio = leadtime_df[columnas_numericas].mean(axis=1) / 30  # Convertir a meses
    lt_std = leadtime_df[columnas_numericas].std(axis=1) / 30  # Convertir a meses

    return pd.DataFrame({
        'SKU': leadtime_df['SKU'],
        'LeadTime_Promedio': lt_promedio,
        'LeadTime_Std': lt_std
    })

def optimizar_inventarios(demanda_stats, leadtime_stats, resultados_combinados):
    """
    Calcula niveles óptimos de inventario
    """
    # Combinar todas las estadísticas
    df = pd.merge(resultados_combinados, demanda_stats, on='SKU')
    df = pd.merge(df, leadtime_stats, on='SKU')

    # Calcular Z-score basado en nivel de servicio
    df['Z_Score'] = df['Nivel_Servicio'].apply(lambda x: stats.norm.ppf(x))

    # Calcular Stock de Seguridad en días
    df['SsDias'] = (
        df['Z_Score'] *
        np.sqrt(
            df['LeadTime_Promedio'] * df['Demanda_Std']**2 +
            df['Demanda_Promedio']**2 * df['LeadTime_Std']**2 +
            0.5/30 * df['Demanda_Std']**2
        ) / df['Demanda_Promedio'] * 30
    )

    # Calcular Techo de inventario en días
    df['TechoDias'] = df['SsDias'] + df['LeadTime_Promedio'] * 30

    # Calcular Nivel óptimo de inventario en días
    df['OptimoDias'] = df['SsDias'] + (df['LeadTime_Promedio'] * 30) / 2

    return df

def mostrar_resultados_finales(df):
    """
    Muestra los resultados finales del análisis
    """
    # Seleccionar columnas relevantes
    columnas_mostrar = [
        'SKU', 'Categoria_ABC_XYZ', 'Nivel_Servicio',
        'SsDias', 'TechoDias', 'OptimoDias'
    ]

    # Formatear números
    resultado_df = df[columnas_mostrar].copy()
    for col in ['SsDias', 'TechoDias', 'OptimoDias']:
        resultado_df[col] = resultado_df[col].round(1)

    print("\nPrimeros 10 SKUs del resultado final:")
    print(resultado_df.head(10).to_string())

    print("\nEstadísticas resumen por categoría ABC-XYZ:")
    resumen = df.groupby('Categoria_ABC_XYZ').agg({
        'SsDias': 'mean',
        'TechoDias': 'mean',
        'OptimoDias': 'mean',
        'SKU': 'count'
    }).round(1)

    print(resumen)

    return resultado_df

def main():
    # URL del archivo
    url = "https://github.com/santiagonajera/Clasificacion_ABC_Python/raw/main/ventas-2024-forecast.xlsx"

    # Cargar y limpiar datos
    lead_time, historico, forecast, precios = load_and_clean_data(url)

    # Realizar clasificaciones ABC-XYZ
    resultados_abc = clasificacion_abc(forecast, precios)
    resultados_xyz = clasificacion_xyz(historico)
    resultados_combinados = combinar_clasificaciones(resultados_abc, resultados_xyz)

    # Calcular estadísticas
    demanda_stats = calcular_estadisticas_demanda(historico)
    leadtime_stats = calcular_estadisticas_leadtime(lead_time)

    # Optimizar inventarios
    resultados_finales = optimizar_inventarios(
        demanda_stats,
        leadtime_stats,
        resultados_combinados
    )

    # Mostrar resultados y obtener DataFrame final
    resultado_df = mostrar_resultados_finales(resultados_finales)

    return resultado_df

if __name__ == "__main__":
    resultados = main()


Primeros 10 SKUs del resultado final:
      SKU Categoria_ABC_XYZ  Nivel_Servicio  SsDias  TechoDias  OptimoDias
0   SKU 2                AX            0.95     8.2       16.3        12.3
1   SKU 3                AX            0.95    11.8       22.0        16.9
2   SKU 4                AX            0.95     8.5       17.6        13.1
3  SKU 12                AY            0.90    10.2       20.3        15.3
4  SKU 11                AX            0.95    10.3       18.8        14.5
5  SKU 14                AY            0.90    11.2       20.9        16.0
6  SKU 18                AZ            0.85     9.1       20.5        14.8
7  SKU 17                AY            0.90    11.2       21.8        16.5
8  SKU 13                AY            0.90     9.9       19.5        14.7
9   SKU 1                AY            0.90     6.4       14.4        10.4

Estadísticas resumen por categoría ABC-XYZ:
                   SsDias  TechoDias  OptimoDias  SKU
Categoria_ABC_XYZ                    